# Логгирование

## Конфигурация логгера

В python есть модуль [`logging`](https://docs.python.org/3/howto/logging.html). Этот модуль даёт инструмент для красивой записи логов, как в файл, так и в терминал. 

Конфигурация определяет, какая информация будет отражена в логе. 

Уровни логов располагаются в следующей последовательности (чем ниже уровень, тем круче его нрав)

* DEBUG 
* INFO 
* WARNING 
* CRITICAL
* ERROR 

In [ ]:
import logging 

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

Например, здесь

```python
level=logging.INFO
```

в конфиге указано, что будут отображаться сообщения уровня `INFO` и ниже, а именно,

* `WARNING`
* `ERROR`
* `CRITICAL`

При этом уровень `DEBUG` отображаться не будет.

Дальше идёт формат

```python
format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
```

который указывает форматирование лога, здесь указано, что сначала будет указана дата и время, затем имя логгера, уровень лога и само сообщение. 

Формат логгирования можно создавать под себя, не обязательно он должен быть только таким.

Для каждого из уровней есть соответствующий метод

In [ ]:
logging.debug('This is a debug message')
logging.info('This is an info message')
logging.warning('This is a warning message')
logging.error('This is an error message')
logging.critical('This is a critical message')

Дата и время в начале сообщений мы видим как раз из-за формата, который мы прописали в конфиге.

## Класс с логгером

Давайте создадим класс, внутри которого будет его личный логгер. 

In [ ]:
class Point:
    point_type = "2D"
    
    def __init__(self, x: float, y: float) -> None:
        self._logger = logging.getLogger("point_logger")
        
        self.x = x 
        self.y = y 

        self._logger.info("Point was created successfully!")
        self._logger.debug("Debug information!")
    
    def get_coordinates(self) -> tuple:
        return (self.x, self.y)

При создании логгера, его имя можно написать статической строкой, например "point_logger". 

In [ ]:
point = Point(0, 0)

Так как у нас выставлен уровень логгирования: `INFO`, то уровень `DEBUG` показан не будет. 

Так бывает, что в ходе рефакторинга классы переменовываются, и в нашем случае имя логгера тоже придётся менять руками. Что лень и не всегда можно заметить. Поэтому есть более автоматический способ, как именовать логгер. 

In [ ]:
class Point:
    point_type = "2D"
    
    def __init__(self, x: float, y: float) -> None:
        self._logger = logging.getLogger(self.__class__.__name__)
        
        self.x = x 
        self.y = y 

        self._logger.info("Point was created successfully!")
        self._logger.debug("Debug information!")
    
    def get_coordinates(self) -> tuple:
        return (self.x, self.y)

In [ ]:
point = Point(0, 0)

В этом случае логгер будет всегда иметь имя класса. Если вы поменяете имя класса, то и имя логгера изменится. Удобно! 

По умолчанию логгер будет писать в стандартный вывод/терминал, но можно перенаправить его, например, в файл. 

## "Как бы" приватность

В питоне есть договорённость: если хочешь показать, что аттрибут приватный, то его имя должно начинаться с нижнего подчёркивания. Но при этом, к нему всё ещё можно будет обратиться снаружи, так что это скорее признак хорошего тона. 

Так как логгер - это штука внутренняя, и у каждого класса обычно есть свой логгер. То хорошей практикой является делать его как бы "приватным" 

```python
self._logger = getLogger(self.__class__.__name__)
```

In [ ]:
point._logger.info("Hello!")

Если хочется чуть больше приватности, то можно добавить 2 нижних подчёркивания. Тогда уже так просто не получится обратиться к аттрибуту извне. 

Но в этом случае делать так не рекомендую, так как в Python по-настоящему ничего нельзя спрятать (ограничить доступ). И 2 нижних подчёркивания будут только сбивать с толку людей, читающих ваш код. 

In [ ]:
class Point:
    def __init__(self, x: float, y: float) -> None:
        self.__logger = logging.getLogger(self.__class__.__name__)
        self.x = x 
        self.y = y 

        self.__logger.info("Point was created successfully!")
    
    def get_coordinates(self) -> tuple:
        area = self.__calculate_area(self)
        return (self.x, self.y, area)

In [ ]:
point = Point(0, 0)

point.__logger.info("Hello!")

НО! Главное помнить, что в python НЕТ ПРИВАТНОСТИ! Если очень захотеть, то всё равно можно обратиться ^____^ 

In [ ]:
point._Point__logger.info("Hello")